In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import scipy.stats as stats
import math

!pip install sklearn
os.getcwd()

You should consider upgrading via the '/homes/iws/bhimar/cse481ds-mental-health/venv/bin/python3 -m pip install --upgrade pip' command.


'/homes/iws/bhimar/cse481ds-mental-health/notebooks/county_level/Regression Analysis'

In [2]:
df = pd.read_csv('../../../data/processed_data/full_df_positive_mental_provider.csv')
df.head()

,State FIPS Code,County FIPS Code,5-digit FIPS Code,State Abbreviation,Name,Release Year,County Ranked (Yes=1/No=0),Premature death raw value,Premature death numerator,Premature death denominator,...,April Average Precipitation,May Average Precipitation,June Average Precipitation,July Average Precipitation,August Average Precipitation,September Average Precipitation,October Average Precipitation,November Average Precipitation,December Average Precipitation,RUCC
0,1,1,1001,AL,Autauga County,2011,1.0,9967.4,675.0,NaN,...,4.19,1.93,3.39,4.38,1.75,4.46,0.98,4.63,3.96,2.0
1,1,3,1003,AL,Baldwin County,2011,1.0,8321.8,2219.0,NaN,...,1.28,1.68,3.15,8.73,2.05,9.89,0.15,3.22,2.78,3.0
2,1,15,1015,AL,Calhoun County,2011,1.0,11719.6,2106.0,NaN,...,5.30,1.65,4.53,3.83,0.93,6.85,0.90,5.00,4.86,3.0
3,1,43,1043,AL,Cullman County,2011,1.0,9564.7,1245.0,NaN,...,9.50,2.30,3.76,5.03,1.51,10.39,0.75,5.95,6.09,4.0
4,1,55,1055,AL,Etowah County,2011,1.0,11812.3,1905.0,NaN,...,6.53,2.77,4.79,4.81,1.53,10.14,0.61,5.69,6.24,3.0


## Preprocess Data

### Filter out NaN cols and rows

In [3]:
identifying_fields = ['State Abbreviation', 'Name', 'Release Year']
with open("factors.txt") as f:
     factors = f.read().splitlines()
with open("outcomes.txt") as f:
     outcomes = f.read().splitlines()
filtered_df = df[identifying_fields + factors + outcomes]
print(filtered_df.shape[0])
filtered_df.head(n=20)
for col in factors + outcomes:
    print(col + "-->" + str(filtered_df[col].isna().sum()))
print()
filtered_df = filtered_df.dropna()
print(filtered_df.shape[0])
for col in factors + outcomes:
    print(col + "-->" + str(filtered_df[col].isna().sum()))

4471
High school graduation raw value-->21
Unemployment raw value-->0
Some college raw value-->0
Ratio of population to mental health providers-->0
Median household income raw value-->0
Average Temperature-->0
January Average Temperature-->0
February Average Temperature-->0
March Average Temperature-->0
April Average Temperature-->0
May Average Temperature-->0
June Average Temperature-->0
July Average Temperature-->0
August Average Temperature-->0
September Average Temperature-->0
October Average Temperature-->0
November Average Temperautre-->0
December Average Temperature-->0
Average Precipitation-->0
January Average Precipitation-->0
February Average Precipitation-->0
March Average Precipitation-->0
April Average Precipitation-->0
May Average Precipitation-->0
June Average Precipitation-->0
July Average Precipitation-->0
August Average Precipitation-->0
September Average Precipitation-->0
October Average Precipitation-->0
November Average Precipitation-->0
December Average Precipitat

### Split into Train and Test

In [4]:
from sklearn.utils import shuffle
filtered_df = shuffle(filtered_df)
X_train = filtered_df[factors][:math.ceil(filtered_df.shape[0] * 0.8)]
X_test = filtered_df[factors][-1 * math.floor(filtered_df.shape[0] * 0.2):]
print(filtered_df.shape[0])
print(X_train.shape[0])
print(X_test.shape[0])
print(pd.merge(X_train, X_test)) # expect likely empty
print()

Y_train = filtered_df[outcomes][:math.ceil(filtered_df.shape[0] * 0.8)]
Y_test = filtered_df[outcomes][-1 * math.floor(filtered_df.shape[0] * 0.2):]
print(filtered_df.shape[0])
print(Y_train.shape[0])
print(Y_test.shape[0])
print()

4441
3553
888
Empty DataFrame
Columns: [High school graduation raw value, Unemployment raw value, Some college raw value, Ratio of population to mental health providers, Median household income raw value, Average Temperature, January Average Temperature, February Average Temperature, March Average Temperature, April Average Temperature, May Average Temperature, June Average Temperature, July Average Temperature, August Average Temperature, September Average Temperature, October Average Temperature, November Average Temperautre, December Average Temperature, Average Precipitation, January Average Precipitation, February Average Precipitation, March Average Precipitation, April Average Precipitation, May Average Precipitation, June Average Precipitation, July Average Precipitation, August Average Precipitation, September Average Precipitation, October Average Precipitation, November Average Precipitation, December Average Precipitation, RUCC]
Index: []

[0 rows x 32 columns]

4441
3553
8

### Save Train and Test as csv files to be read into regression modules

In [7]:
X_train.to_csv("X_train.csv", index=False)
Y_train.to_csv("Y_train.csv", index=False)
X_test.to_csv("X_test.csv", index=False)
Y_test.to_csv("Y_test.csv", index=False)